## Imports

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
import numpy as np
import os, sys
import torch
import random
import wandb  # Import the Weights & Biases library

from torch.utils.data import TensorDataset, random_split
import torch.nn as nn
from sklearn.metrics import confusion_matrix
import sys

In [3]:
sys.path.append("/Users/naliniramanathan/projects/snn_project/randman")
import randman
from randman import Randman

## Universal Model Settings

We set them up here since some of the random manifold components are dependent on this so it has the right structure

In [4]:
# Settings
nb_inputs  = 100
nb_hidden  = 4
nb_outputs = 2

time_step = 1e-3
nb_steps  = 200

batch_size = 128*nb_outputs


In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

dtype = torch.float

In [6]:
tau_mem = 10e-3
tau_syn = 5e-3

alpha   = float(np.exp(-time_step/tau_syn))
beta    = float(np.exp(-time_step/tau_mem))

weight_scale = 7*(1.0-beta) # this should give us some spikes to begin with

w1 = torch.empty((nb_inputs, nb_hidden),  device=device, dtype=dtype, requires_grad=True)
w2 = torch.empty((nb_hidden, nb_outputs), device=device, dtype=dtype, requires_grad=True)
v1 = torch.empty((nb_hidden, nb_hidden), device=device, dtype=dtype, requires_grad=True)

## Create the Random Manifolds

In [7]:
def standardize(x,eps=1e-7):
    mi,_ = x.min(0)
    ma,_ = x.max(0)
    return (x-mi)/(ma-mi+eps)

def make_spiking_dataset(nb_classes=10, nb_units=100, nb_steps=100, step_frac=1.0, dim_manifold=2, nb_spikes=1, nb_samples=1000, alpha=2.0, shuffle=True, classification=True, seed=None):
    """ Generates event-based generalized spiking randman classification/regression dataset. 
    In this dataset each unit fires a fixed number of spikes. So ratebased or spike count based decoding won't work. 
    All the information is stored in the relative timing between spikes.
    For regression datasets the intrinsic manifold coordinates are returned for each target.
    Args: 
        nb_classes: The number of classes to generate
        nb_units: The number of units to assume
        nb_steps: The number of time steps to assume
        step_frac: Fraction of time steps from beginning of each to contain spikes (default 1.0)
        nb_spikes: The number of spikes per unit
        nb_samples: Number of samples from each manifold per class
        alpha: Randman smoothness parameter
        shuffe: Whether to shuffle the dataset
        classification: Whether to generate a classification (default) or regression dataset
        seed: The random seed (default: None)
    Returns: 
        A tuple of data,labels. The data is structured as numpy array 
        (sample x event x 2 ) where the last dimension contains 
        the relative [0,1] (time,unit) coordinates and labels.
    """
  
    data = []
    labels = []
    targets = []

    if seed is not None:
        np.random.seed(seed)
    
    max_value = np.iinfo(np.int64).max
    randman_seeds = np.random.randint(max_value, size=(nb_classes,nb_spikes) )

    for k in range(nb_classes):
        x = np.random.rand(nb_samples,dim_manifold)
        submans = [ randman.Randman(nb_units, dim_manifold, alpha=alpha, seed=randman_seeds[k,i]) for i in range(nb_spikes) ]
        units = []
        times = []
        for i,rm in enumerate(submans):
            y = rm.eval_manifold(x)
            y = standardize(y)
            units.append(np.repeat(np.arange(nb_units).reshape(1,-1),nb_samples,axis=0))
            times.append(y.numpy())

        units = np.concatenate(units,axis=1)
        times = np.concatenate(times,axis=1)
        events = np.stack([times,units],axis=2)
        data.append(events)
        labels.append(k*np.ones(len(units)))
        targets.append(x)

    data = np.concatenate(data, axis=0)
    labels = np.array(np.concatenate(labels, axis=0), dtype=np.int64)
    targets = np.concatenate(targets, axis=0)

    if shuffle:
        idx = np.arange(len(data))
        np.random.shuffle(idx)
        data = data[idx]
        labels = labels[idx]
        targets = targets[idx]

    data[:,:,0] *= nb_steps*step_frac
    # data = np.array(data, dtype=int)

    if classification:
        return data, labels
    else:
        return data, targets

### Data Preparation for Model Training

In [8]:
# Convert the spike events to a tensor format suitable for our SNN PyTorch implementation

def convert_to_x_data(spike_events, nb_steps, nb_units):
    # Initialize a tensor of zeros (nb_steps, nb_units)
    x_data = torch.zeros((nb_steps, nb_units), dtype=torch.float32)
    
    # Iterate through each spike event (time, unit)
    for event in spike_events:
        time_step = int(event[0])  # Convert time to an index in the range [0, nb_steps-1]
        unit_id = int(event[1])  # Unit index (neuron)
        x_data[time_step, unit_id] = 1  # Mark the spike at the correct time step and unit
    
    return x_data

def create_x_data(data):
    x_data = []
    for sample in data:
        x_data.append(convert_to_x_data(sample, nb_steps, nb_units=nb_inputs))

    # Convert list of tensors into a single PyTorch tensor (batch of samples)
    x_data = torch.stack(x_data)  # Shape: (num_samples, nb_steps, nb_units)
    return x_data

In [9]:
# x_data = create_x_data(data)

In [10]:
# data_tensor = torch.tensor(x_data, dtype=torch.float32)  # Shape: (num_samples, num_events, 2)
# labels_tensor = torch.tensor(labels, dtype=torch.int64)  # Shape: (num_samples,)

# # Combine data and labels into a PyTorch Dataset
# dataset = TensorDataset(data_tensor, labels_tensor)

# total_size = len(dataset)
# subset_size = total_size // 3

# # Ensure that the sizes add up correctly, handling potential remainder
# remaining_size = total_size - 2 * subset_size

# # Split the dataset into train, validation, and test subsets
# train_subset, val_subset, test_subset = random_split(dataset, [subset_size, subset_size, remaining_size])

# # Convert back to x_data and y_data for train subset
# x_train = [data for data, _ in train_subset]
# x_train = torch.stack(x_train).to(device)
# y_train = [label for _, label in train_subset]
# y_train = torch.tensor(y_train, dtype=torch.int64).to(device)

# # Convert back to x_data and y_data for validation subset
# x_val = [data for data, _ in val_subset]
# x_val = torch.stack(x_val).to(device)
# y_val = [label for _, label in val_subset]
# y_val = torch.tensor(y_val, dtype=torch.int64).to(device)

# # Convert back to x_data and y_data for test subset
# x_test = [data for data, _ in test_subset]
# x_test = torch.stack(x_test).to(device)
# y_test = [label for _, label in test_subset]
# y_test = torch.tensor(y_test, dtype=torch.int64).to(device)

# # Now you have x_train, y_train, x_val, y_val, x_test, y_test
# print(f"Train set size: {len(train_subset)}")
# print(f"Validation set size: {len(val_subset)}")
# print(f"Test set size: {len(test_subset)}")
# print(f"x_train shape: {x_train.shape}, y_train shape: {y_train.shape}")
# print(f"x_val shape: {x_val.shape}, y_val shape: {y_val.shape}")
# print(f"x_test shape: {x_test.shape}, y_test shape: {y_test.shape}")

In [11]:
# data_id = 2
# plt.imshow(x_test[data_id].cpu().t(), cmap=plt.cm.gray_r, aspect="auto")
# plt.xlabel("Time (ms)")
# plt.ylabel("Unit")
# sns.despine()

## Create a Baseline SNN Hybrid Model

### Model Implementation - No Recurrent Weights

The input is a set of spikes. At each timestep, certain SNN neurons fire in response to those inputs. The ANNs act as recurrent units within the network. We'll exclude the recurrent weights to simplify things to start.

#### SNN Only

In [12]:
def spike_fn(x):
    out = torch.zeros_like(x)
    out[x > 0] = 1.0
    return out

def SNN(inputs, timesteps=nb_steps):
    inputs = inputs.to(device)
    h1 = torch.einsum("abc,cd->abd", (inputs, w1))
    syn = torch.zeros((batch_size,nb_hidden), device=device, dtype=dtype)
    mem = torch.zeros((batch_size,nb_hidden), device=device, dtype=dtype)

    mem_rec = []
    spk_rec = []

    # Compute hidden layer activity
    for t in range(timesteps):
        mthr = mem-1.0
        out = spike_fn(mthr)
        rst = out.detach() # We do not want to backprop through the reset

        new_syn = alpha*syn +h1[:,t]
        new_mem = (beta*mem +syn)*(1.0-rst)

        mem_rec.append(mem)
        spk_rec.append(out)
        
        mem = new_mem
        syn = new_syn

    mem_rec = torch.stack(mem_rec,dim=1)
    spk_rec = torch.stack(spk_rec,dim=1)

    # Readout layer
    h2= torch.einsum("abc,cd->abd", (spk_rec, w2))
    flt = torch.zeros((batch_size,nb_outputs), device=device, dtype=dtype)
    out = torch.zeros((batch_size,nb_outputs), device=device, dtype=dtype)
    out_rec = [out]
    for t in range(timesteps):
        new_flt = alpha*flt +h2[:,t]
        new_out = beta*out +flt

        flt = new_flt
        out = new_out

        out_rec.append(out)

    out_rec = torch.stack(out_rec,dim=1)
    other_recs = [mem_rec, spk_rec]
    return out_rec, other_recs

In [13]:
def plot_voltage_traces(mem, spk=None, dim=(3,5), spike_height=5):
    gs=GridSpec(*dim)
    if spk is not None:
        dat = 1.0*mem
        dat[spk>0.0] = spike_height
        dat = dat.detach().cpu().numpy()
    else:
        dat = mem.detach().cpu().numpy()
    for i in range(np.prod(dim)):
        if i==0: a0=ax=plt.subplot(gs[i])
        else: ax=plt.subplot(gs[i],sharey=a0)
        ax.plot(dat[i])
        ax.axis("off")

##### SNN Regularization

##### SNN Recurrent

In [15]:
def SNN_recurrent(inputs, timesteps=nb_steps):
    inputs = inputs.to(device)
    syn = torch.zeros((batch_size,nb_hidden), device=device, dtype=dtype)
    mem = torch.zeros((batch_size,nb_hidden), device=device, dtype=dtype)
    mem_rec = []
    spk_rec = []
    out = torch.zeros((batch_size, nb_hidden), device=device, dtype=dtype)
    h1_from_input = torch.einsum("abc,cd->abd", (inputs, w1))
    # Compute hidden layer activity
    for t in range(timesteps):
        h1 = h1_from_input[:,t] + torch.einsum("ab,bc->ac", (out, v1))
        mthr = mem-1.0
        out = spike_fn(mthr)
        rst = out.detach() # We do not want to backprop through the reset

        new_syn = alpha*syn +h1
        new_mem = (beta*mem +syn)*(1.0-rst)

        mem_rec.append(mem)
        spk_rec.append(out)
        
        mem = new_mem
        syn = new_syn

    mem_rec = torch.stack(mem_rec,dim=1)
    spk_rec = torch.stack(spk_rec,dim=1)

    # Readout layer
    h2= torch.einsum("abc,cd->abd", (spk_rec, w2))
    flt = torch.zeros((batch_size,nb_outputs), device=device, dtype=dtype)
    out = torch.zeros((batch_size,nb_outputs), device=device, dtype=dtype)
    out_rec = [out]
    for t in range(timesteps):
        new_flt = alpha*flt +h2[:,t]
        new_out = beta*out +flt

        flt = new_flt
        out = new_out

        out_rec.append(out)

    out_rec = torch.stack(out_rec,dim=1)
    other_recs = [mem_rec, spk_rec]
    return out_rec, other_recs

#### ANN Only

Let's model an RNN with a similar structure to our SNN. It will take in our temporal data (spikes), and then transform it accordingly in each layer using the weights.

Since we take spikes, weight them, and then output them using traditional tanh, we don't really need to worry about conversion.

We'd usually have recurrent weights for the RNN, but since we excluded them for the SNN we'll do the same here.


In [16]:
def ANN_non_recurrent_with_LIF_output(inputs, timesteps=nb_steps):
    inputs = inputs.to(device)
    
    # Weights for the hidden layer for RNN is just w1 -- multiplying by inputs just gives us the output at each timestep
    h1 = torch.einsum("abc,cd->abd", (inputs, w1))
    # We could also add a bias term but let's exclude for now to make the merging more simple

    interim_rec = []

    # The difference for a pure ANN is that the hidden and the output layer usually happen at the same time
    # It's really fine that they're different here though if we just compute them sequentially

    for t in range(timesteps):

        layer_weights = h1[:,t]

        # Use a tanh function similar to what's done in other models
        out = torch.tanh(layer_weights)
        interim_rec.append(out)

    interim_rec = torch.stack(interim_rec,dim=1)
    # We will use the same LIF model as before, and just use the raw output as an input to start since it has a window of 1

    # Readout layer
    h2= torch.einsum("abc,cd->abd", (interim_rec, w2))
    flt = torch.zeros((batch_size,nb_outputs), device=device, dtype=dtype)
    out = torch.zeros((batch_size,nb_outputs), device=device, dtype=dtype)
    out_rec = [out]

    # Convert the h2 readout layer to a rate that can be transmitted to the output layer
    
    for t in range(nb_steps):
        new_flt = alpha*flt +h2[:,t]
        new_out = beta*out +flt

        flt = new_flt
        out = new_out

        out_rec.append(out)

    out_rec = torch.stack(out_rec,dim=1)
    other_recs = interim_rec
    return out_rec, other_recs

##### With Recurrent Network

In [17]:
def RNN_with_LIF_output(inputs, timesteps=nb_steps):
    inputs = inputs.to(device)

    # Weights for the hidden layer for RNN is just w1 -- multiplying by inputs just gives us the output at each timestep
    h1 = torch.einsum("abc,cd->abd", (inputs, w1))
    # We could also add a bias term but let's exclude for now to make the merging more simple

    interim_rec = []
    
    # The difference for a pure ANN is that the hidden and the output layer usually happen at the same time
    # It's really fine that they're different here though if we just compute them sequentially
    out = torch.zeros((batch_size, nb_hidden), device=device, dtype=dtype)

    for t in range(timesteps):

        layer_weights = h1[:,t] + torch.einsum("ab,bc->ac", (out, v1))

        # Use a tanh function similar to what's done in other models
        out = torch.tanh(layer_weights)
        interim_rec.append(out)

    interim_rec = torch.stack(interim_rec,dim=1)
    # We will use the same LIF model as before, and just use the raw output as an input to start since it has a window of 1

    # Readout layer
    h2= torch.einsum("abc,cd->abd", (interim_rec, w2))
    flt = torch.zeros((batch_size,nb_outputs), device=device, dtype=dtype)
    out = torch.zeros((batch_size,nb_outputs), device=device, dtype=dtype)
    out_rec = [out]

    # Convert the h2 readout layer to a rate that can be transmitted to the output layer
    
    for t in range(nb_steps):
        new_flt = alpha*flt +h2[:,t]
        new_out = beta*out +flt

        flt = new_flt
        out = new_out

        out_rec.append(out)

    out_rec = torch.stack(out_rec,dim=1)
    other_recs = interim_rec
    return out_rec, other_recs

#### ANN and SNN - Network to Network

For this implementation, we just have 1 timestep, so we don't need to worry about converting ANN to SNN or any of the intermediary weights.

In [18]:
torch.manual_seed(42)
snn_mask = torch.randint(0, 2, (nb_hidden,), dtype=torch.float32, device=device)  # Randomly assign spiking or analog neurons
snn_mask # 1 is spiking

tensor([1., 1., 0., 0.], device='mps:0')

Options for SNN to ANN conversion include:
1. Time of first spike (this makes less sense with a window of size one)
2. Rate (this also makes less sense with a window of size one)
3. ISI (it's a difference between spikes so I think this also makes less sense)
4. Just take it and weight it (learnable) --> trying this first

In [19]:
def Hybrid_ANN_non_recurrent(inputs, timesteps=nb_steps):
    inputs = inputs.to(device)
    # Weight matrix for training, I think this can be used for both SNN and ANN but the training may have to be different
    h1 = torch.einsum("abc,cd->abd", (inputs, w1))
    # Let's just do 2 matrices to keep it clean for now
    h1_ann = h1.clone() * (1.0-snn_mask)  # ANN neurons
    h1_snn = h1.clone() * snn_mask  # SNN neurons
    syn = torch.zeros((batch_size,nb_hidden), device=device, dtype=dtype)
    mem = torch.zeros((batch_size,nb_hidden), device=device, dtype=dtype)

    mem_rec = []
    spk_rec = []
    ann_rec = []

    # Compute hidden layer activity
    for t in range(timesteps):

        # SNN neurons

        # Apply the mask to the synaptic input to ensure SNN neurons are only updated with spiking rules
        mem = mem*snn_mask
        syn = syn*snn_mask

        mthr = mem-1.0
        out_snn = spike_fn(mthr)
        rst = out_snn.detach() # We do not want to backprop through the reset

        new_syn = alpha*syn +h1_snn[:,t]
        new_mem = (beta*mem +syn)*(1.0-rst)

        mem_rec.append(mem)
        spk_rec.append(out_snn)

        # ANN neurons - don't interact with the synaptic input at all to start
        out_ann = torch.tanh(h1_ann[:,t])
        ann_rec.append(out_ann)
        
        mem = new_mem
        syn = new_syn

    mem_rec = torch.stack(mem_rec,dim=1)
    spk_rec = torch.stack(spk_rec,dim=1)
    ann_rec = torch.stack(ann_rec,dim=1)

    h2_snn = torch.einsum("abc,cd->abd", (spk_rec, w2))
    h2_ann= torch.einsum("abc,cd->abd", (ann_rec, w2))

    # We can add the two together to get the final output since the two do not interact at all
    h2 = h2_snn + h2_ann

    flt = torch.zeros((batch_size,nb_outputs), device=device, dtype=dtype)
    out = torch.zeros((batch_size,nb_outputs), device=device, dtype=dtype)
    out_rec = [out]
    for t in range(timesteps):
        new_flt = alpha*flt +h2[:,t]
        new_out = beta*out +flt

        flt = new_flt
        out = new_out

        out_rec.append(out)

    out_rec = torch.stack(out_rec,dim=1)
    other_recs = [mem_rec, spk_rec, ann_rec]
    return out_rec, other_recs

##### With Recurrent Network

In [20]:
def Hybrid_RNN(inputs, timesteps=nb_steps):
    inputs = inputs.to(device)
    # Weight matrix for training, I think this can be used for both SNN and ANN but the training may have to be different
    h1 = torch.einsum("abc,cd->abd", (inputs, w1))
    # Let's just do 2 matrices to keep it clean for now
    h1_ann = h1.clone() * (1.0-snn_mask)  # ANN neurons
    h1_snn = h1.clone() * snn_mask  # SNN neurons
    syn = torch.zeros((batch_size,nb_hidden), device=device, dtype=dtype)
    mem = torch.zeros((batch_size,nb_hidden), device=device, dtype=dtype)

    mem_rec = []
    spk_rec = []
    ann_rec = []

    out_ann = torch.zeros((batch_size, nb_hidden), device=device, dtype=dtype)
    
    # Compute hidden layer activity
    for t in range(timesteps):

        # SNN neurons

        # Apply the mask to the synaptic input to ensure SNN neurons are only updated with spiking rules
        mem = mem*snn_mask
        syn = syn*snn_mask

        mthr = mem-1.0
        out_snn = spike_fn(mthr)
        rst = out_snn.detach() # We do not want to backprop through the reset

        new_syn = alpha*syn +h1_snn[:,t]
        new_mem = (beta*mem +syn)*(1.0-rst)

        mem_rec.append(mem)
        spk_rec.append(out_snn)

        # ANN neurons - don't interact with the synaptic input at all to start
        out_ann = torch.tanh(h1_ann[:,t]) + torch.einsum("ab,bc->ac", (out_ann, v1))
        ann_rec.append(out_ann)
        
        mem = new_mem
        syn = new_syn

    mem_rec = torch.stack(mem_rec,dim=1)
    spk_rec = torch.stack(spk_rec,dim=1)
    ann_rec = torch.stack(ann_rec,dim=1)

    h2_snn = torch.einsum("abc,cd->abd", (spk_rec, w2))
    h2_ann= torch.einsum("abc,cd->abd", (ann_rec, w2))

    # We can add the two together to get the final output since the two do not interact at all
    h2 = h2_snn + h2_ann

    flt = torch.zeros((batch_size,nb_outputs), device=device, dtype=dtype)
    out = torch.zeros((batch_size,nb_outputs), device=device, dtype=dtype)
    out_rec = [out]
    for t in range(timesteps):
        new_flt = alpha*flt +h2[:,t]
        new_out = beta*out +flt

        flt = new_flt
        out = new_out

        out_rec.append(out)

    out_rec = torch.stack(out_rec,dim=1)
    other_recs = [mem_rec, spk_rec, ann_rec]
    return out_rec, other_recs

##### Recurrent SNN Network

In [21]:
def Hybrid_RNN_SNN_rec(inputs, timesteps=nb_steps):
    inputs = inputs.to(device)
    # Weight matrix for training, I think this can be used for both SNN and ANN but the training may have to be different
    h1 = torch.einsum("abc,cd->abd", (inputs, w1))
    # Let's just do 2 matrices to keep it clean for now
    h1_ann = h1.clone() * (1.0-snn_mask)  # ANN neurons
    h1_snn_input = h1.clone() * snn_mask  # SNN neurons
    syn = torch.zeros((batch_size,nb_hidden), device=device, dtype=dtype)
    mem = torch.zeros((batch_size,nb_hidden), device=device, dtype=dtype)

    mem_rec = []
    spk_rec = []
    ann_rec = []

    out_ann = torch.zeros((batch_size, nb_hidden), device=device, dtype=dtype)
    out_snn = torch.zeros((batch_size, nb_hidden), device=device, dtype=dtype)
    # Compute hidden layer activity
    for t in range(timesteps):
        h1_snn = h1_snn_input[:,t] + torch.einsum("ab,bc->ac", (out_snn, v1))
        # SNN neurons

        # Apply the mask to the synaptic input to ensure SNN neurons are only updated with spiking rules
        mem = mem*snn_mask
        syn = syn*snn_mask

        mthr = mem-1.0
        out_snn = spike_fn(mthr)
        rst = out_snn.detach() # We do not want to backprop through the reset

        new_syn = alpha*syn +h1_snn
        new_mem = (beta*mem +syn)*(1.0-rst)

        mem_rec.append(mem)
        spk_rec.append(out_snn)

        # ANN neurons - don't interact with the synaptic input at all to start
        out_ann = torch.tanh(h1_ann[:,t]) + torch.einsum("ab,bc->ac", (out_ann, v1))
        ann_rec.append(out_ann)
        
        mem = new_mem
        syn = new_syn

    mem_rec = torch.stack(mem_rec,dim=1)
    spk_rec = torch.stack(spk_rec,dim=1)
    ann_rec = torch.stack(ann_rec,dim=1)

    h2_snn = torch.einsum("abc,cd->abd", (spk_rec, w2))
    h2_ann= torch.einsum("abc,cd->abd", (ann_rec, w2))

    # We can add the two together to get the final output since the two do not interact at all
    h2 = h2_snn + h2_ann

    flt = torch.zeros((batch_size,nb_outputs), device=device, dtype=dtype)
    out = torch.zeros((batch_size,nb_outputs), device=device, dtype=dtype)
    out_rec = [out]
    for t in range(timesteps):
        new_flt = alpha*flt +h2[:,t]
        new_out = beta*out +flt

        flt = new_flt
        out = new_out

        out_rec.append(out)

    out_rec = torch.stack(out_rec,dim=1)
    other_recs = [mem_rec, spk_rec, ann_rec]
    return out_rec, other_recs

### ANN and SNN - Layer to Layer

In [22]:
def Hybrid_RNN_SNN_V1_same_layer(inputs, timesteps=nb_steps):
    inputs = inputs.to(device)
    # Weight matrix for training, I think this can be used for both SNN and ANN but the training may have to be different
    h1 = torch.einsum("abc,cd->abd", (inputs, w1))
    # Let's just do 2 matrices to keep it clean for now
    h1_ann = h1.clone() * (1.0-snn_mask)  # ANN neurons
    h1_snn_input = h1.clone() * snn_mask  # SNN neurons
    syn = torch.zeros((batch_size,nb_hidden), device=device, dtype=dtype)
    mem = torch.zeros((batch_size,nb_hidden), device=device, dtype=dtype)

    mem_rec = []
    spk_rec = []
    ann_rec = []
    # (O_A + O_S) * W_2 = (O_A * W_2 + O_S * W_2) by distributive property, no theoretical change there apart from H1
    # Prove/think about this after more after test it
    out_ann = torch.zeros((batch_size, nb_hidden), device=device, dtype=dtype)
    out_snn = torch.zeros((batch_size, nb_hidden), device=device, dtype=dtype)
    # Compute hidden layer activity
    for t in range(timesteps):
        out = out_ann + out_snn
        h1_snn = h1_snn_input[:,t] + torch.einsum("ab,bc->ac", (out, v1))
        # SNN neurons

        # Apply the mask to the synaptic input to ensure SNN neurons are only updated with spiking rules
        mem = mem*snn_mask
        syn = syn*snn_mask

        mthr = mem-1.0
        out_snn = spike_fn(mthr)
        rst = out_snn.detach() # We do not want to backprop through the reset

        new_syn = alpha*syn +h1_snn
        new_mem = (beta*mem +syn)*(1.0-rst)

        mem_rec.append(mem)
        spk_rec.append(out_snn)

        # ANN neurons - don't interact with the synaptic input at all to start
        out_ann = torch.tanh(h1_ann[:,t]) + torch.einsum("ab,bc->ac", (out, v1))
        ann_rec.append(out_ann)
        
        mem = new_mem
        syn = new_syn

    mem_rec = torch.stack(mem_rec,dim=1)
    spk_rec = torch.stack(spk_rec,dim=1)
    ann_rec = torch.stack(ann_rec,dim=1)

    h2_snn = torch.einsum("abc,cd->abd", (spk_rec, w2))
    h2_ann = torch.einsum("abc,cd->abd", (ann_rec, w2))

    # We can add the two together to get the final output since the two do not interact at all
    h2 = h2_snn + h2_ann

    flt = torch.zeros((batch_size,nb_outputs), device=device, dtype=dtype)
    out = torch.zeros((batch_size,nb_outputs), device=device, dtype=dtype)
    out_rec = [out]
    for t in range(timesteps):
        new_flt = alpha*flt +h2[:,t]
        new_out = beta*out +flt

        flt = new_flt
        out = new_out

        out_rec.append(out)

    out_rec = torch.stack(out_rec,dim=1)
    other_recs = [mem_rec, spk_rec, ann_rec]
    return out_rec, other_recs

### With Training (Surrogate Gradient)

In [23]:
class SurrGradSpike(torch.autograd.Function):
    """
    Here we implement our spiking nonlinearity which also implements 
    the surrogate gradient. By subclassing torch.autograd.Function, 
    we will be able to use all of PyTorch's autograd functionality.
    Here we use the normalized negative part of a fast sigmoid 
    as this was done in Zenke & Ganguli (2018).
    """
    
    scale = 100.0 # controls steepness of surrogate gradient

    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we compute a step function of the input Tensor
        and return it. ctx is a context object that we use to stash information which 
        we need to later backpropagate our error signals. To achieve this we use the 
        ctx.save_for_backward method.
        """
        ctx.save_for_backward(input)
        out = torch.zeros_like(input)
        out[input > 0] = 1.0
        return out

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor we need to compute the 
        surrogate gradient of the loss with respect to the input. 
        Here we use the normalized negative part of a fast sigmoid 
        as this was done in Zenke & Ganguli (2018).
        """
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad = grad_input/(SurrGradSpike.scale*torch.abs(input)+1.0)**2
        return grad
    
# here we overwrite our naive spike function by the "SurrGradSpike" nonlinearity which implements a surrogate gradient
spike_fn = SurrGradSpike.apply

In [24]:
def compute_classification_accuracy(x_data, y_data, model=SNN):
    """ 
    Computes classification accuracy and confusion matrix on supplied data in batches.
    
    Returns:
        accuracy: Overall classification accuracy.
        conf_matrix: Confusion matrix of shape (num_classes, num_classes).
    """
    all_preds = []
    all_labels = []

    output,recs = model(x_data)
    spike_count = 0
    m,_=torch.max(output,1)
    if "Hybrid" or "SNN" in model.__name__:
        spks = recs[1]
        spks[spks>0.0] = 1.0
        spks[spks<0.0] = 0.0
        spike_count = torch.sum(spks).detach().cpu().numpy()
    # Compute training accuracy
    _,am=torch.max(m,1)
    acc = np.mean((y_data==am).detach().cpu().numpy())
    all_preds.extend(am.cpu().numpy())
    all_labels.extend(y_data.cpu().numpy())
    
    # Compute overall accuracy

    # Compute confusion matrix
    conf_matrix = confusion_matrix(all_labels, all_preds)
    

    return acc, conf_matrix, spike_count
    
def plot_confusion_matrix(conf_matrix, class_names, title):
    """ Plots the confusion matrix. """
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title(f"{title} Confusion Matrix - Random Manifolds")
    plt.show()

In [25]:
def train_and_print_results(model, x_train, y_train, x_val, y_val, x_test, y_test, epochs=100, regularization=False, lr=2e-3, l1=1e-5, l2=1e-5, wandb_run=None):
    global w1, w2, v1
    # The following lines will reinitialize the weights at the start of each run
    with torch.no_grad():
        torch.nn.init.normal_(w1, mean=0.0, std=weight_scale/np.sqrt(nb_inputs))
        torch.nn.init.normal_(w2, mean=0.0, std=weight_scale/np.sqrt(nb_hidden))
        torch.nn.init.normal_(v1, mean=0.0, std=weight_scale/np.sqrt(nb_hidden))

    print("init done")

    params = [w1, w2, v1]
    optimizer = torch.optim.Adam(params, lr=lr, betas=(0.9, 0.999))
    log_softmax_fn = nn.LogSoftmax(dim=1)
    loss_fn = nn.NLLLoss()
    loss_hist_train = []
    loss_hist_val = []
    val_accuracy_hist = []
    spike_count_train_epoch = 0
    spike_count_val_epoch = 0

    for e in range(epochs):
        # --- Training Phase ---
        output_train, other_recs_train = model(x_train)
        m,_=torch.max(output_train,1)
        log_p_y_train = log_softmax_fn(m)
        loss_val_train = loss_fn(log_p_y_train, y_train)

        if regularization:
            spks = other_recs_train[1]
            reg_loss = l1 * torch.sum(spks)
            reg_loss += l2 * torch.mean(torch.sum(torch.sum(spks, dim=0), dim=0)**2)
            loss_val_train += reg_loss

        optimizer.zero_grad()
        loss_val_train.backward()
        optimizer.step()
        loss_hist_train.append(loss_val_train.item())

        if "SNN" in model.__name__ or "Hybrid" in model.__name__:
            spks_train_epoch = torch.sum(other_recs_train[1][other_recs_train[1] >= 1.0]).detach().cpu().numpy()

        # --- Validation Phase ---
        with torch.no_grad():
            output_val, other_recs_val = model(x_val)
            m,_=torch.max(output_val,1)

            log_p_y_val = log_softmax_fn(m)
            loss_val_val = loss_fn(log_p_y_val, y_val)

            val_accuracy, _, _ = compute_classification_accuracy(x_val, y_val, model=model)
            val_accuracy_hist.append(val_accuracy)
            loss_hist_val.append(loss_val_val.item())

            if "SNN" in model.__name__ or "Hybrid" in model.__name__:
                spike_count_val_epoch = torch.sum(other_recs_val[1][other_recs_val[1] >= 1.0]).detach().cpu().numpy()

            # Log metrics to Wandb
            if wandb_run:
                wandb_run.log({"train_loss": loss_val_train.item(),
                               "val_loss": loss_val_val.item(),
                               "val_accuracy": val_accuracy,
                               "epoch": e})
                if "SNN" in model.__name__ or "Hybrid" in model.__name__:
                    wandb_run.log({"spike_count_train_epoch": spike_count_train_epoch,
                                   "spike_count_val_epoch": spike_count_val_epoch})

    # Evaluation after training loop
    train_accuracy, train_conf_matrix, _ = compute_classification_accuracy(x_train, y_train, model=model)
    test_accuracy, test_conf_matrix, _ = compute_classification_accuracy(x_test, y_test, model=model)
    val_accuracy_final, val_conf_matrix, _ = compute_classification_accuracy(x_val, y_val, model=model)

    print(f"Final Train Accuracy: {train_accuracy * 100:.2f}%")
    print(f"Final Validation Accuracy: {val_accuracy_final * 100:.2f}%")
    print(f"Final Test Accuracy: {test_accuracy * 100:.2f}%")

    if wandb_run:
        wandb_run.log({"final_train_accuracy": train_accuracy,
                       "final_test_accuracy": test_accuracy,
                       "final_val_accuracy": val_accuracy})

    return val_accuracy # For training sweeps

### Train with wandb

In [26]:
function_mappings = {
    'SNN': SNN,
    'SNN_recurrent': SNN_recurrent,
    'ANN_non_recurrent_with_LIF_output': ANN_non_recurrent_with_LIF_output,
    'RNN_with_LIF_output': RNN_with_LIF_output,
    'Hybrid_ANN_non_recurrent': Hybrid_ANN_non_recurrent,
    'Hybrid_RNN': Hybrid_RNN,
    'Hybrid_RNN_SNN_rec': Hybrid_RNN_SNN_rec,
    'Hybrid_RNN_SNN_V1_same_layer': Hybrid_RNN_SNN_V1_same_layer
}

In [27]:
# x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

In [28]:
# # Define the number of runs
# num_runs = 5
# all_test_accuracies = []

# # Define your hyperparameters
# config = {
#     "epochs": 150,
#     "regularization": True,
#     "l1": 5e-6,
#     "l2": 1e-6,
#     "learning_rate": 2e-3,
#     "optimizer": "Adam",
#     "model_name": "SNN" # Add a name for grouping
# }

# # Assume x_data and labels are defined globally or loaded here
# # Example (replace with your actual data loading):

# for run_idx in range(num_runs):
#     # Set a different random seed for each run
#     seed = 42 + run_idx
#     torch.manual_seed(seed)
#     np.random.seed(seed)
#     random.seed(seed)
#     data,labels = make_spiking_dataset(nb_classes=nb_outputs, nb_units=nb_inputs, nb_steps=nb_steps, dim_manifold=1, seed=seed, nb_samples=int(batch_size/nb_outputs)*3)
    
#     x_data = create_x_data(data)    
#     # Reinitialize the dataset and split it
#     data_tensor = torch.tensor(x_data, dtype=torch.float32)
#     labels_tensor = torch.tensor(labels, dtype=torch.int64)
#     dataset = TensorDataset(data_tensor, labels_tensor)

#     total_size = len(dataset)
#     third_size = total_size // 3  # Integer division to get the base size

#     train_size = third_size
#     val_size = third_size
#     remaining_size = total_size - train_size - val_size
#     test_size = remaining_size  # The rest of the data goes to the test set

#     train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

#     x_train = torch.stack([data for data, _ in train_dataset]).to(device)
#     y_train = torch.tensor([label for _, label in train_dataset], dtype=torch.int64).to(device)
#     x_val = torch.stack([data for data, _ in val_dataset]).to(device)
#     y_val = torch.tensor([label for _, label in val_dataset], dtype=torch.int64).to(device)
#     x_test = torch.stack([data for data, _ in test_dataset]).to(device)
#     y_test = torch.tensor([label for _, label in test_dataset], dtype=torch.int64).to(device)

#     # Initialize Wandb run
#     wandb_run = wandb.init(project="SNN_test",
#                             name=f"{config['model_name']}_run_{run_idx}",
#                             config=config,
#                             group=config['model_name'])

#     model = function_mappings[config["model_name"]]
#     # Train and get results for the current run
#     test_accuracy = train_and_print_results(model, x_train, y_train, x_val, y_val, x_test, y_test,
#                                             epochs=config["epochs"],
#                                             regularization=config["regularization"],
#                                             lr=config["learning_rate"],
#                                             l1=config["l1"],
#                                             l2=config["l2"],
#                                             wandb_run=wandb_run)

#     all_test_accuracies.append(test_accuracy)
#     # Finish the Wandb run for this initialization
#     wandb_run.finish()

# # Print overall average results
# print("\n--- Average Results Across Runs ---")
# print(f"Average Test Accuracy: {np.mean(all_test_accuracies) * 100:.2f}%")


In [29]:
# def model_wandb(num_runs, epochs, model):
#     all_test_accuracies = []

#     # Define your hyperparameters
#     config = {
#         "epochs": epochs,
#         "regularization": True,
#         "l1": 5e-6,
#         "l2": 1e-6,
#         "learning_rate": 2e-3,
#         "optimizer": "Adam",
#         "model_name": model # Add a name for grouping
#     }

#     # Assume x_data and labels are defined globally or loaded here
#     # Example (replace with your actual data loading):

#     for run_idx in range(num_runs):
#         # Set a different random seed for each run
#         seed = 42 + run_idx
#         torch.manual_seed(seed)
#         np.random.seed(seed)
#         random.seed(seed)
#         data,labels = make_spiking_dataset(nb_classes=nb_outputs, nb_units=nb_inputs, nb_steps=nb_steps, dim_manifold=1, seed=seed, nb_samples=int(batch_size/nb_outputs)*3)
        
#         x_data = create_x_data(data)    
#         # Reinitialize the dataset and split it
#         data_tensor = torch.tensor(x_data, dtype=torch.float32)
#         labels_tensor = torch.tensor(labels, dtype=torch.int64)
#         dataset = TensorDataset(data_tensor, labels_tensor)

#         total_size = len(dataset)
#         third_size = total_size // 3  # Integer division to get the base size

#         train_size = third_size
#         val_size = third_size
#         remaining_size = total_size - train_size - val_size
#         test_size = remaining_size  # The rest of the data goes to the test set

#         train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

#         x_train = torch.stack([data for data, _ in train_dataset]).to(device)
#         y_train = torch.tensor([label for _, label in train_dataset], dtype=torch.int64).to(device)
#         x_val = torch.stack([data for data, _ in val_dataset]).to(device)
#         y_val = torch.tensor([label for _, label in val_dataset], dtype=torch.int64).to(device)
#         x_test = torch.stack([data for data, _ in test_dataset]).to(device)
#         y_test = torch.tensor([label for _, label in test_dataset], dtype=torch.int64).to(device)

#         # Initialize Wandb run
#         wandb_run = wandb.init(project="SNN_test",
#                                 name=f"{config['model_name']}_run_{run_idx}",
#                                 config=config,
#                                 group=config['model_name'])

#         model = function_mappings[config["model_name"]]
#         # Train and get results for the current run
#         test_accuracy = train_and_print_results(model, x_train, y_train, x_val, y_val, x_test, y_test,
#                                                 epochs=config["epochs"],
#                                                 regularization=config["regularization"],
#                                                 l1=config["l1"],
#                                                 l2=config["l2"],
#                                                 wandb_run=wandb_run)

#         all_test_accuracies.append(test_accuracy)
#         # Finish the Wandb run for this initialization
#         wandb_run.finish()

#     # Print overall average results
#     print("\n--- Average Results Across Runs ---")
#     print(f"Average Test Accuracy: {np.mean(all_test_accuracies) * 100:.2f}%")


In [30]:
# # Now repeat the process for the other models
# for model_name, model_func in function_mappings.items():
#     model_wandb(num_runs=5, epochs=150, model=model_name)

### Regularization Optimization

In [31]:
def train_sweep():
    # Initialize a Wandb run
    with wandb.init(project="SNN_test_optimize", config=wandb.config) as run:
        config = wandb.config
        model_name = config['model_name']
        run.name = f"{model_name}-l1_{config['l1']}-l2_{config['l2']}"
        data, labels = make_spiking_dataset(nb_classes=nb_outputs, nb_units=nb_inputs, nb_steps=nb_steps, dim_manifold=1, seed=42, nb_samples=int(batch_size/nb_outputs)*3)
        # Load your data (ensure nb_outputs, device are defined)
        x_data = create_x_data(data)    
        # Reinitialize the dataset and split it
        data_tensor = torch.tensor(x_data, dtype=torch.float32)
        labels_tensor = torch.tensor(labels, dtype=torch.int64)
        dataset = TensorDataset(data_tensor, labels_tensor)

        total_size = len(dataset)
        third_size = total_size // 3  # Integer division to get the base size

        train_size = third_size
        val_size = third_size
        remaining_size = total_size - train_size - val_size
        test_size = remaining_size  # The rest of the data goes to the test set

        train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

        x_train = torch.stack([data for data, _ in train_dataset]).to(device)
        y_train = torch.tensor([label for _, label in train_dataset], dtype=torch.int64).to(device)
        x_val = torch.stack([data for data, _ in val_dataset]).to(device)
        y_val = torch.tensor([label for _, label in val_dataset], dtype=torch.int64).to(device)
        x_test = torch.stack([data for data, _ in test_dataset]).to(device)
        y_test = torch.tensor([label for _, label in test_dataset], dtype=torch.int64).to(device)

        # Instantiate your model (assuming Hybrid_RNN_SNN_rec is defined globally)
        model = function_mappings[config["model_name"]]

        # Train the model
        final_val_accuracy = train_and_print_results(model, x_train, y_train, x_val, y_val, x_test, y_test,epochs=config['epochs'], regularization=config['regularization'], wandb_run=run, l1=config['l1'], l2=config['l2'], lr=config['learning_rate'])

In [32]:
# # Now let's do the sweep for regularization optimization (SNN models only)
# sweep_config = {
#     'method': 'grid',
#     'name': 'SNN Regularization Sweep',
#     'metric': {
#         'name': 'val_accuracy',
#         'goal': 'maximize'
#     },
#     'parameters': {
#         'l1': {
#             'values': [1e-6, 5e-6, 1e-5]
#         },
#         'l2': {
#             'values': [1e-6, 5e-6, 1e-5]
#         },
#         'learning_rate': {
#             'value': 2e-3 # Keep learning rate constant for this sweep
#         },
#         'epochs': {
#             'value': 150 # Keep epochs constant for this sweep
#         },
#         'regularization':{
#             'value': True # Ensure regularization is always on for this sweep
#         },
#         'optimizer': {
#             'value': "Adam"
#         },
#         'model_name':{
#             'value': "SNN"
#         }
#     }
# }
# sweep_id = wandb.sweep(sweep_config, project="SNN_test_optimize")

# wandb.agent(sweep_id, train_sweep, count=9) # Run all 3 x 3 combinations


In [33]:
# Now let's do it for all models
sweep_config = {
    'method': 'grid',
    'name': 'SNN Regularization Sweep',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'l1': {
            'values': [1e-6, 5e-6, 1e-5]
        },
        'l2': {
            'values': [1e-6, 5e-6, 1e-5]
        },
        'learning_rate': {
            'value': 2e-3 # Keep learning rate constant for this sweep
        },
        'epochs': {
            'value': 150 # Keep epochs constant for this sweep
        },
        'regularization':{
            'value': True # Ensure regularization is always on for this sweep
        },
        'optimizer': {
            'value': "Adam"
        },
        'model_name':{
            'value': "SNN"
        }
    }
}
sweep_config['parameters']['model_name']['value'] = 'Hybrid_RNN_SNN_V1_same_layer'  # Change to the desired model name

# Initialize a Wandb sweep for the current model
sweep_id = wandb.sweep(sweep_config, project="SNN_test_optimize")

# Run the sweep agent
wandb.agent(sweep_id, train_sweep, count=9)  # Run all 3 x 3 combinations

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: zfqum03h
Sweep URL: https://wandb.ai/nar8991-new-york-university/SNN_test_optimize/sweeps/zfqum03h


wandb: Agent Starting Run: j0qz4hl4 with config:
wandb: 	epochs: 150
wandb: 	l1: 1e-06
wandb: 	l2: 1e-06
wandb: 	learning_rate: 0.002
wandb: 	model_name: Hybrid_RNN_SNN_V1_same_layer
wandb: 	optimizer: Adam
wandb: 	regularization: True
wandb: Currently logged in as: nar8991 (nar8991-new-york-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/var/folders/7b/0zqwym191dq_5jvhhbf5lfgr0000gn/T/ipykernel_69979/3981686502.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data_tensor = torch.tensor(x_data, dtype=torch.float32)


init done
Final Train Accuracy: 89.45%
Final Validation Accuracy: 79.69%
Final Test Accuracy: 78.52%


...

/var/folders/7b/0zqwym191dq_5jvhhbf5lfgr0000gn/T/ipykernel_69979/3981686502.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data_tensor = torch.tensor(x_data, dtype=torch.float32)


init done


wandb: Network error (ConnectionError), entering retry loop.


Final Train Accuracy: 64.84%
Final Validation Accuracy: 63.67%
Final Test Accuracy: 65.62%


epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇██
final_test_accuracy,▁
final_train_accuracy,▁
final_val_accuracy,▁
spike_count_train_epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
spike_count_val_epoch,█▆▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▇▄▃▁▁▃▃▃▃▃▃▃▃▃▃▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇█████
val_loss,▁▁▇██▇▇▆▆▅▄▄▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,149
final_test_accuracy,0.65625


In [ ]:
# Now let's do it for all models
sweep_config = {
    'method': 'grid',
    'name': 'SNN Regularization Sweep',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'l1': {
            'values': [1e-6, 5e-6, 1e-5]
        },
        'l2': {
            'values': [1e-6, 5e-6, 1e-5]
        },
        'learning_rate': {
            'value': 2e-3 # Keep learning rate constant for this sweep
        },
        'epochs': {
            'value': 150 # Keep epochs constant for this sweep
        },
        'regularization':{
            'value': True # Ensure regularization is always on for this sweep
        },
        'optimizer': {
            'value': "Adam"
        },
        'model_name':{
            'value': "SNN"
        }
    }
}

for model_name, model_func in function_mappings.items():
    # sweep_config['name'] = f'Regularization Sweep - {model_name}'
    if model_name == "SNN":
        continue
    if model_name == "SNN_recurrent":
        continue
    if model_name == "ANN_non_recurrent_with_LIF_output":
        continue
    # Set the model name in the sweep config
    sweep_config['parameters']['model_name']['value'] = model_name

    # Initialize a Wandb sweep for the current model
    sweep_id = wandb.sweep(sweep_config, project="SNN_test_optimize")

    # Run the sweep agent
    wandb.agent(sweep_id, train_sweep, count=9)  # Run all 3 x 3 combinations

Create sweep with ID: kg3cw864
Sweep URL: https://wandb.ai/nar8991-new-york-university/SNN_test_optimize/sweeps/kg3cw864


wandb: Agent Starting Run: 5lwektnr with config:
wandb: 	epochs: 150
wandb: 	l1: 1e-06
wandb: 	l2: 1e-06
wandb: 	learning_rate: 0.002
wandb: 	model_name: RNN_with_LIF_output
wandb: 	optimizer: Adam
wandb: 	regularization: True


/var/folders/7b/0zqwym191dq_5jvhhbf5lfgr0000gn/T/ipykernel_69979/3981686502.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data_tensor = torch.tensor(x_data, dtype=torch.float32)


init done
Final Train Accuracy: 100.00%
Final Validation Accuracy: 100.00%
Final Test Accuracy: 100.00%


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇██
final_test_accuracy,▁
final_train_accuracy,▁
final_val_accuracy,▁
train_loss,█▇▇▆▆▆▆▅▅▃▃▃▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▂▄▅▅▅▅▅▆▆▇▇▇▇▇▇▇███████████████████████
val_loss,█▆▆▆▆▅▅▄▄▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,149
final_test_accuracy,1
final_train_accuracy,1
final_val_accuracy,1


...

init done
Final Train Accuracy: 64.84%
Final Validation Accuracy: 63.67%
Final Test Accuracy: 65.62%


epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇███
final_test_accuracy,▁
final_train_accuracy,▁
final_val_accuracy,▁
spike_count_train_epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
spike_count_val_epoch,█▆▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▆▆▅▃▁▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇████
val_loss,▁▂█▆▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
epoch,149
final_test_accuracy,0.65625


wandb: Agent Starting Run: k25tlpat with config:
wandb: 	epochs: 150
wandb: 	l1: 5e-06
wandb: 	l2: 1e-06
wandb: 	learning_rate: 0.002
wandb: 	model_name: Hybrid_RNN_SNN_V1_same_layer
wandb: 	optimizer: Adam
wandb: 	regularization: True


/var/folders/7b/0zqwym191dq_5jvhhbf5lfgr0000gn/T/ipykernel_69979/3981686502.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data_tensor = torch.tensor(x_data, dtype=torch.float32)


init done
